**Mounting the google drive on the Colab session**

In [ ]:
#Mounting gdrive to gain access to the dataset
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Installing pyLDAvis module for visualizing the model output and performance**

In [ ]:
!pip install pyLDAvis

In [ ]:
#to get rid of the warning messages in the output
import warnings
warnings.filterwarnings("ignore")

**Importing necessary libraries**

In [ ]:
#Importing necessary libraries
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import numpy as np
import nltk
nltk.download('wordnet')
import pandas as pd
import json
import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
pip install pandas --upgrade

**Importing data and cleaning it off the unwanted columns**

In [ ]:
#importing data
data = pd.read_json('/content/gdrive/My Drive/cran_d.json')
data.drop(['author', 'bibliography','title'], axis = 1)

,id,body
0,1,experimental investigation of the aerodynamics...
1,2,simple shear flow past a flat plate in an inco...
2,3,the boundary layer in simple shear flow past a...
3,4,approximate solutions of the incompressible la...
4,5,one-dimensional transient heat conduction into...
...,...,...
1395,1396,shear buckling of clamped and simply-supported...
1396,1397,critical shear stress of an infinitely long si...
1397,1398,stability of rectangular plates under shear an...
1398,1399,buckling of transverse stiffened plates under ...


**Data Preprocessing (Stemming, Lemmatizing and Tokenizing)**

In [ ]:
#Data Preprocessing
stemmer = SnowballStemmer(language='english')
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
processed_data = data['body'].map(preprocess)
processed_data

0       [experiment, investig, aerodynam, wing, slipst...
1       [simpl, shear, flow, past, flat, plate, incomp...
2       [boundari, layer, simpl, shear, flow, past, fl...
3       [approxim, solut, incompress, laminar, boundar...
4       [dimension, transient, heat, conduct, doubl, l...
                              ...                        
1395    [shear, buckl, clamp, simpli, support, infinit...
1396    [critic, shear, stress, infinit, long, simpli,...
1397    [stabil, rectangular, plat, shear, bend, forc,...
1398    [buckl, transvers, stiffen, plat, shear, paper...
1399    [buckl, shear, stress, simpli, support, infini...
Name: body, Length: 1400, dtype: object

**Creating dictionary and corpus**

In [ ]:
#Creating dictionary and the word corpus
dictionary = corpora.Dictionary(processed_data)
corpus = [dictionary.doc2bow(text) for text in processed_data]

**Creating tf-idf model object on ‘bow_corpus’**

In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.053302445840307376),
 (1, 0.08206383857277204),
 (2, 0.048917665629352944),
 (3, 0.06499483304247398),
 (4, 0.0783847211538762),
 (5, 0.028340962543825454),
 (6, 0.04495784529265081),
 (7, 0.07385116512341265),
 (8, 0.08119993508064181),
 (9, 0.0630256806900624),
 (10, 0.509307940076426),
 (11, 0.04000670522109575),
 (12, 0.1544567313753436),
 (13, 0.03512348375093592),
 (14, 0.04937572128417836),
 (15, 0.09791849493987904),
 (16, 0.1552670353827112),
 (17, 0.09486618233401206),
 (18, 0.055570484690629675),
 (19, 0.07350588940046916),
 (20, 0.01687521477063807),
 (21, 0.048194417622460475),
 (22, 0.049287002591385444),
 (23, 0.22862026072668198),
 (24, 0.05840643038238435),
 (25, 0.12333639554404417),
 (26, 0.03512348375093592),
 (27, 0.03157347976088682),
 (28, 0.2254914465151426),
 (29, 0.050427687185562314),
 (30, 0.050819352648683),
 (31, 0.08250683576792811),
 (32, 0.038005863257038426),
 (33, 0.07482919458418888),
 (34, 0.095596222296088),
 (35, 0.040268032190421986),
 (36

**Building the model with number of topics(n) = 10**

In [ ]:
lda1 = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=15, workers=4)
for idx, topic in lda1.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.019*"buckl" + 0.019*"shell" + 0.011*"load" + 0.010*"stress" + 0.010*"cylind" + 0.008*"stiffen" + 0.008*"cylindr" + 0.008*"bend" + 0.007*"plat" + 0.007*"elast"
Topic: 1 Word: 0.004*"suction" + 0.003*"balanc" + 0.002*"coolant" + 0.002*"discret" + 0.002*"exchang" + 0.002*"hodograph" + 0.002*"ionize" + 0.002*"tollmien" + 0.002*"hoff" + 0.002*"sweat"
Topic: 2 Word: 0.013*"creep" + 0.009*"column" + 0.008*"fatigu" + 0.004*"throat" + 0.003*"life" + 0.003*"collaps" + 0.003*"grind" + 0.003*"galerkin" + 0.003*"stress" + 0.003*"sampl"
Topic: 3 Word: 0.007*"rough" + 0.004*"corridor" + 0.003*"flame" + 0.003*"molecul" + 0.003*"trip" + 0.003*"lattic" + 0.003*"hypothesi" + 0.002*"entranc" + 0.002*"modul" + 0.002*"depth"
Topic: 4 Word: 0.004*"round" + 0.003*"cruciform" + 0.003*"torsion" + 0.003*"devot" + 0.002*"shield" + 0.002*"fin" + 0.002*"freestream" + 0.002*"ablat" + 0.002*"planar" + 0.002*"reinforc"
Topic: 5 Word: 0.007*"slip" + 0.003*"oval" + 0.003*"hoop" + 0.002*"knudsen" + 0.002

**Performance Evaluation for n=10**

In [ ]:
#Computing Perplexity
print('\nPerplexity: ', lda1.log_perplexity(corpus))

# Computing Coherence Score
coherence_lda1 = CoherenceModel(model=lda1, texts=processed_data, dictionary=dictionary, coherence='c_v')
coherence_lda11 = coherence_lda1.get_coherence()
print('\nCoherence Score: ', coherence_lda11)


Perplexity:  -7.03095840369207

Coherence Score:  0.6559759112426058


**Visualizing the n=10 model**

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda1, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.263697 -0.042741       1        1  89.234026
0     -0.055294  0.136902       2        1   5.581375
3      0.041820 -0.016088       3        1   0.872385
2      0.042276  0.005316       4        1   0.790302
1      0.039638 -0.018030       5        1   0.691922
4      0.040618 -0.009930       6        1   0.663832
6      0.042791 -0.012132       7        1   0.596699
5      0.038833 -0.015580       8        1   0.559054
9      0.038247 -0.013497       9        1   0.550900
7      0.034768 -0.014221      10        1   0.459507, topic_info=        Term        Freq       Total Category  logprob  loglift
559   cylind  334.000000  334.000000  Default  30.0000  30.0000
29      load  228.000000  228.000000  Default  29.0000  29.0000
445    buckl  170.000000  170.000000  Default  28.0000  28.0000
355   stress  203.000000  203.000000  Default  27.0000  27.0000
1283   shell  156.000000  156.000000  Default  26.0000  26.0000
...      ...         ...         ...      ...      ...      ...
2002    code    0.864964   10.771597  Topic10  -6.4325   2.8608
1223   hover    0.867778   20.135534  Topic10  -6.4293   2.2385
1222   grind    0.928086   29.696097  Topic10  -6.3621   1.9171
1171  shield    0.660069    6.289025  Topic10  -6.7028   3.1286
1628   water    0.696635   25.482173  Topic10  -6.6489   1.7833

[453 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1155      1  0.795584     ablat
1155      2  0.151540     ablat
1155      6  0.018942     ablat
1744      1  0.426978  accommod
1744      2  0.213489  accommod
...     ...       ...       ...
3769      9  0.261638  withdraw
3187      1  0.378219    wrinkl
3187      2  0.189109    wrinkl
3187      3  0.189109    wrinkl
3187     10  0.189109    wrinkl

[1226 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 1, 4, 3, 2, 5, 7, 6, 10, 8])

**Building model with n=12**

In [ ]:
lda2 = gensim.models.LdaMulticore(corpus_tfidf, num_topics=12, id2word=dictionary, passes=15, workers=4)
for idx, topic in lda2.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.004*"pohlhausen" + 0.004*"fluctuat" + 0.004*"freon" + 0.004*"nois" + 0.004*"airfoil" + 0.003*"major" + 0.003*"contour" + 0.003*"volum" + 0.002*"separ" + 0.002*"wrinkl"
Topic: 1 Word: 0.022*"buckl" + 0.018*"shell" + 0.011*"cylind" + 0.011*"panel" + 0.011*"load" + 0.010*"stress" + 0.009*"creep" + 0.008*"bend" + 0.008*"cylindr" + 0.008*"stiffen"
Topic: 2 Word: 0.003*"eddi" + 0.003*"orbit" + 0.002*"sodium" + 0.002*"sectori" + 0.002*"lee" + 0.002*"structur" + 0.002*"solar" + 0.002*"degreek" + 0.002*"galcit" + 0.002*"thicken"
Topic: 3 Word: 0.007*"layer" + 0.007*"boundari" + 0.007*"heat" + 0.006*"flow" + 0.006*"shock" + 0.005*"solut" + 0.005*"temperatur" + 0.005*"transfer" + 0.005*"equat" + 0.004*"pressur"
Topic: 4 Word: 0.004*"charg" + 0.004*"particl" + 0.003*"disk" + 0.002*"absolut" + 0.002*"advers" + 0.002*"retard" + 0.002*"propel" + 0.002*"vtol" + 0.002*"transport" + 0.002*"pound"
Topic: 5 Word: 0.004*"entropi" + 0.004*"creep" + 0.003*"column" + 0.003*"capac" + 0.003*"mo

**Performance Evaluation on n=12**

In [ ]:
#Computing Perplexity
print('\nPerplexity: ', lda2.log_perplexity(corpus))

# Computing Coherence Score
coherence_lda2 = CoherenceModel(model=lda2, texts=processed_data, dictionary=dictionary, coherence='c_v')
coherence_lda21 = coherence_lda2.get_coherence()
print('\nCoherence Score: ', coherence_lda21)


Perplexity:  -7.069868914753971

Coherence Score:  0.5650497922089681


**Visualization of model with n=12**

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda2, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.252873 -0.003342       1        1  71.611901
8      0.153822  0.075523       2        1  15.454899
1      0.046952 -0.159976       3        1   6.017846
11    -0.040977  0.016712       4        1   1.293930
0     -0.007424  0.030529       5        1   1.207259
10    -0.062203  0.004037       6        1   1.063711
7     -0.059032  0.014068       7        1   0.918106
4     -0.051738  0.005403       8        1   0.659845
2     -0.061890  0.005078       9        1   0.524552
5     -0.057011 -0.000560      10        1   0.509579
6     -0.053403  0.006902      11        1   0.398331
9     -0.059967  0.005627      12        1   0.340042, topic_info=            Term        Freq       Total Category  logprob  loglift
52          wing  432.000000  432.000000  Default  30.0000  30.0000
559       cylind  345.000000  345.000000  Default  29.0000  29.0000
445        buckl  183.000000  183.000000  Default  28.0000  28.0000
1283       shell  184.000000  184.000000  Default  27.0000  27.0000
355       stress  224.000000  224.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
1023       sound    0.915492   32.099517  Topic12  -6.0746   2.1267
1422  background    0.652212   15.671784  Topic12  -6.4137   2.5046
1001     absolut    0.482984    9.105966  Topic12  -6.7141   2.7472
1634       denot    0.452432    6.872221  Topic12  -6.7795   2.9633
999         roll    0.468703   22.521020  Topic12  -6.7441   1.8116

[602 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
1155      1  0.787338  ablat
1155      2  0.017116  ablat
1155      3  0.136928  ablat
1155      8  0.017116  ablat
1155      9  0.017116  ablat
...     ...       ...    ...
441       9  0.060680   zone
2351      1  0.206741   zoom
2351      2  0.206741   zoom
2351      3  0.206741   zoom
2351      9  0.206741   zoom

[2265 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 9, 2, 12, 1, 11, 8, 5, 3, 6, 7, 10])

**Building model with n = 14, computing the metrics and visualizing the performance**

In [ ]:
lda3 = gensim.models.LdaMulticore(corpus_tfidf, num_topics=14, id2word=dictionary, passes=15, workers=4)
for idx, topic in lda3.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.008*"sandwich" + 0.005*"orthotrop" + 0.004*"deton" + 0.004*"isotrop" + 0.003*"unpressur" + 0.002*"schlicht" + 0.002*"diatom" + 0.002*"individu" + 0.002*"honeycomb" + 0.002*"statist"
Topic: 1 Word: 0.010*"propel" + 0.010*"nois" + 0.009*"fatigu" + 0.008*"aircraft" + 0.007*"structur" + 0.005*"tilt" + 0.005*"slipstream" + 0.005*"grind" + 0.005*"acoust" + 0.005*"shallow"
Topic: 2 Word: 0.004*"fin" + 0.003*"lattic" + 0.003*"oge" + 0.003*"fold" + 0.003*"preston" + 0.002*"dodecagon" + 0.002*"sextic" + 0.002*"glide" + 0.002*"million" + 0.002*"skip"
Topic: 3 Word: 0.006*"layer" + 0.006*"wing" + 0.006*"flow" + 0.006*"boundari" + 0.005*"bodi" + 0.005*"heat" + 0.005*"shock" + 0.005*"pressur" + 0.005*"number" + 0.005*"solut"
Topic: 4 Word: 0.017*"creep" + 0.010*"column" + 0.009*"detach" + 0.008*"rough" + 0.006*"plastic" + 0.006*"membran" + 0.005*"collaps" + 0.004*"element" + 0.004*"strain" + 0.003*"mechan"
Topic: 5 Word: 0.015*"inject" + 0.013*"nozzl" + 0.007*"blade" + 0.007*"ablat"

In [ ]:
#Computing Perplexity
print('\nPerplexity: ', lda3.log_perplexity(corpus))

# Computing Coherence Score
coherence_lda3 = CoherenceModel(model=lda3, texts=processed_data, dictionary=dictionary, coherence='c_v')
coherence_lda31 = coherence_lda3.get_coherence()
print('\nCoherence Score: ', coherence_lda31)


Perplexity:  -7.09254873557678

Coherence Score:  0.5743057349193464


In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda3, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.314707  0.026340       1        1  87.856738
5      0.039593 -0.124945       2        1   2.852535
1     -0.011980 -0.016413       3        1   2.296425
4     -0.012580  0.008010       4        1   1.317865
9     -0.032390  0.001635       5        1   0.803169
13    -0.037269  0.009482       6        1   0.781482
6     -0.028948  0.023963       7        1   0.660192
10    -0.039618  0.004516       8        1   0.640387
11    -0.032284  0.017257       9        1   0.617171
0     -0.034602  0.010061      10        1   0.586692
7     -0.036099  0.012118      11        1   0.466361
8     -0.025730  0.009792      12        1   0.429841
2     -0.033965  0.007856      13        1   0.385824
12    -0.028835  0.010327      14        1   0.305317, topic_info=            Term        Freq       Total Category  logprob  loglift
445        buckl  331.000000  331.000000  Default  30.0000  30.0000
417        panel  164.000000  164.000000  Default  29.0000  29.0000
559       cylind  425.000000  425.000000  Default  28.0000  28.0000
1283       shell  318.000000  318.000000  Default  27.0000  27.0000
29          load  318.000000  318.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
159       resist    0.823051   14.593575  Topic14  -6.0734   2.9163
1114  hyperveloc    0.557161    7.049643  Topic14  -6.4635   3.2537
1498       sampl    0.520017    7.125178  Topic14  -6.5325   3.1740
867         unit    0.592636   13.419306  Topic14  -6.4018   2.6717
728         duct    0.512617    8.857159  Topic14  -6.5469   2.9421

[657 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1155      1  0.038058     ablat
1155      2  0.875342     ablat
1155      4  0.038058     ablat
1631      1  0.233013  abstract
1631      7  0.233013  abstract
...     ...       ...       ...
3187      1  0.171333    wrinkl
3187      7  0.171333    wrinkl
3187      9  0.171333    wrinkl
2387      1  0.223426     young
2387      9  0.223426     young

[1776 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 6, 2, 5, 10, 14, 7, 11, 12, 1, 8, 9, 3, 13])

**Building model with n = 16, computing the metrics and visualizing the performance**

In [ ]:
lda4 = gensim.models.LdaMulticore(corpus_tfidf, num_topics=16, id2word=dictionary, passes=15, workers=4)
for idx, topic in lda4.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.003*"latitud" + 0.003*"releas" + 0.003*"blow" + 0.003*"night" + 0.003*"suction" + 0.003*"grid" + 0.003*"overexpand" + 0.002*"confin" + 0.002*"claus" + 0.002*"globul"
Topic: 1 Word: 0.011*"layer" + 0.010*"boundari" + 0.009*"shock" + 0.007*"flow" + 0.007*"number" + 0.007*"turbul" + 0.007*"separ" + 0.006*"pressur" + 0.006*"nozzl" + 0.006*"transit"
Topic: 2 Word: 0.014*"buckl" + 0.014*"shell" + 0.009*"cylind" + 0.009*"load" + 0.008*"stress" + 0.007*"creep" + 0.007*"plate" + 0.006*"solut" + 0.006*"cylindr" + 0.006*"bend"
Topic: 3 Word: 0.003*"probe" + 0.003*"deby" + 0.003*"molecul" + 0.003*"wrinkl" + 0.003*"gust" + 0.003*"modulus" + 0.002*"discharg" + 0.002*"airforc" + 0.002*"membran" + 0.002*"colloc"
Topic: 4 Word: 0.004*"restraint" + 0.003*"deton" + 0.003*"maccol" + 0.003*"heavi" + 0.002*"lattic" + 0.002*"nod" + 0.002*"isobar" + 0.002*"melt" + 0.002*"accumul" + 0.002*"hoff"
Topic: 5 Word: 0.004*"space" + 0.004*"simpli" + 0.003*"stiffen" + 0.003*"oge" + 0.003*"support" + 0

In [ ]:
#Computing Perplexity
print('\nPerplexity: ', lda4.log_perplexity(corpus))

# Computing Coherence Score
coherence_lda4 = CoherenceModel(model=lda4, texts=processed_data, dictionary=dictionary, coherence='c_v')
coherence_lda41 = coherence_lda4.get_coherence()
print('\nCoherence Score: ', coherence_lda41)


Perplexity:  -7.13904302282289

Coherence Score:  0.599452827833622


In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda4, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.219705  0.025254       1        1  35.077720
11    -0.216012  0.055197       2        1  27.657656
6     -0.168638  0.033916       3        1  18.216876
2     -0.161095 -0.082110       4        1  12.132202
7     -0.025200 -0.125990       5        1   3.233867
15     0.076255  0.007480       6        1   0.433446
10     0.070033  0.007484       7        1   0.418496
8      0.073828  0.015863       8        1   0.392888
13     0.075597  0.012745       9        1   0.382843
5      0.069927  0.003896      10        1   0.327398
4      0.074191  0.008164      11        1   0.311123
0      0.072149  0.005434      12        1   0.306908
12     0.072669  0.009898      13        1   0.305155
9      0.071602  0.008754      14        1   0.299499
14     0.068176  0.007725      15        1   0.274361
3      0.066223  0.006290      16        1   0.229561, topic_info=          Term        Freq       Total Category  logprob  loglift
1283     shell  221.000000  221.000000  Default  30.0000  30.0000
445      buckl  239.000000  239.000000  Default  29.0000  29.0000
126       heat  438.000000  438.000000  Default  28.0000  28.0000
52        wing  545.000000  545.000000  Default  27.0000  27.0000
171      field  215.000000  215.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
2830   membran    0.603149   36.031139  Topic16  -6.0990   1.9868
1440  hyperbol    0.471165   18.038602  Topic16  -6.3460   2.4317
1231  hartmann    0.375684   10.125495  Topic16  -6.5725   2.7827
2245   entranc    0.363274    8.607764  Topic16  -6.6060   2.9115
283       tube    0.363348  198.291906  Topic16  -6.6058  -0.2254

[810 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
1155      1  0.019038  ablat
1155      2  0.019038  ablat
1155      3  0.837678  ablat
1155      4  0.019038  ablat
1155      5  0.095191  ablat
...     ...       ...    ...
2351      1  0.198576   zoom
2351      2  0.198576   zoom
2351      3  0.198576   zoom
2351      4  0.198576   zoom
2351     11  0.198576   zoom

[3414 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 12, 7, 3, 8, 16, 11, 9, 14, 6, 5, 1, 13, 10, 15, 4])

**Building model with n = 13, computing the metrics and visualizing the performance**

In [ ]:
lda5 = gensim.models.LdaMulticore(corpus_tfidf, num_topics=13, id2word=dictionary, passes=15, workers=4)
for idx, topic in lda5.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.008*"suction" + 0.006*"navier" + 0.004*"slipstream" + 0.004*"perpendicular" + 0.004*"flexur" + 0.003*"generalis" + 0.003*"fourth" + 0.003*"couett" + 0.003*"discharg" + 0.003*"apex"
Topic: 1 Word: 0.004*"aeroelast" + 0.003*"crocco" + 0.003*"wrinkl" + 0.003*"falkner" + 0.003*"skan" + 0.003*"kernel" + 0.003*"tailplan" + 0.003*"reservoir" + 0.003*"panel" + 0.003*"essenti"
Topic: 2 Word: 0.005*"sting" + 0.003*"propag" + 0.003*"lattic" + 0.003*"flap" + 0.003*"million" + 0.003*"boom" + 0.003*"hing" + 0.003*"hyperbol" + 0.003*"oge" + 0.002*"split"
Topic: 3 Word: 0.009*"fatigu" + 0.005*"turbin" + 0.004*"strain" + 0.004*"right" + 0.003*"cycl" + 0.003*"damag" + 0.003*"cumul" + 0.002*"southwel" + 0.002*"stress" + 0.002*"product"
Topic: 4 Word: 0.008*"stiffen" + 0.006*"simpli" + 0.006*"clamp" + 0.005*"oseen" + 0.004*"support" + 0.004*"plat" + 0.003*"charg" + 0.003*"matrix" + 0.003*"panel" + 0.002*"mise"
Topic: 5 Word: 0.022*"panel" + 0.004*"flutter" + 0.004*"freon" + 0.004*"buckl" 

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda5, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.305744 -0.036628       1        1  86.722166
9      0.043568  0.165147       2        1   6.170165
6     -0.038776 -0.006182       3        1   0.964534
1     -0.039521 -0.018981       4        1   0.907702
0     -0.019728 -0.021452       5        1   0.895086
2     -0.026748 -0.017421       6        1   0.733969
5     -0.029787  0.002250       7        1   0.670708
4     -0.028127 -0.002678       8        1   0.584146
11    -0.036669 -0.013383       9        1   0.552984
3     -0.033229 -0.001333      10        1   0.518616
12    -0.029941 -0.019390      11        1   0.473330
8     -0.033050 -0.013951      12        1   0.416026
10    -0.033735 -0.015997      13        1   0.390568, topic_info=          Term        Freq       Total Category  logprob  loglift
386    flutter  257.000000  257.000000  Default  30.0000  30.0000
445      buckl  189.000000  189.000000  Default  29.0000  29.0000
29        load  262.000000  262.000000  Default  28.0000  28.0000
355     stress  216.000000  216.000000  Default  27.0000  27.0000
1283     shell  174.000000  174.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
417      panel    0.727260   48.931084  Topic13  -6.4434   1.3364
566   recoveri    0.610207   16.907542  Topic13  -6.6188   2.2236
1034  interior    0.572857    9.617104  Topic13  -6.6820   2.7247
1763  turbojet    0.557324    9.237506  Topic13  -6.7095   2.7374
2239     remov    0.540113    5.729155  Topic13  -6.7409   3.1838

[618 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1001      1  0.142104  absolut
1001      2  0.142104  absolut
1001      3  0.142104  absolut
1001      5  0.284207  absolut
1804      1  0.174238  accumul
...     ...       ...      ...
3187      2  0.153960   wrinkl
3187      4  0.461879   wrinkl
2351      1  0.263351     zoom
2351      2  0.263351     zoom
2351     13  0.263351     zoom

[1845 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 10, 7, 2, 1, 3, 6, 5, 12, 4, 13, 9, 11])

**Building model with n = 15, computing the metrics and visualizing the performance**

In [ ]:
lda6 = gensim.models.LdaMulticore(corpus_tfidf, num_topics=15, id2word=dictionary, passes=15, workers=4)
for idx, topic in lda6.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.011*"inject" + 0.011*"transfer" + 0.009*"heat" + 0.008*"layer" + 0.008*"transit" + 0.007*"shock" + 0.007*"number" + 0.007*"turbul" + 0.007*"boundari" + 0.007*"tube"
Topic: 1 Word: 0.003*"heater" + 0.003*"liapunov" + 0.002*"claus" + 0.002*"elaps" + 0.002*"knudsen" + 0.002*"inconsist" + 0.002*"invert" + 0.002*"torqu" + 0.002*"extension" + 0.002*"curtain"
Topic: 2 Word: 0.005*"freon" + 0.003*"comment" + 0.003*"lubric" + 0.002*"plume" + 0.002*"accommod" + 0.002*"nacell" + 0.002*"nod" + 0.002*"sublim" + 0.002*"wrinkl" + 0.002*"gorcum"
Topic: 3 Word: 0.006*"prescrib" + 0.004*"vehicl" + 0.003*"radiat" + 0.003*"mainstream" + 0.002*"matric" + 0.002*"emiss" + 0.002*"particl" + 0.002*"nonlift" + 0.002*"buri" + 0.002*"earth"
Topic: 4 Word: 0.003*"multi" + 0.003*"theodorsen" + 0.002*"vertex" + 0.002*"grid" + 0.002*"status" + 0.002*"magnus" + 0.002*"isobar" + 0.002*"code" + 0.002*"sweptback" + 0.002*"convert"
Topic: 5 Word: 0.005*"exit" + 0.005*"converg" + 0.005*"rocket" + 0.004*"ex

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda6, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.263278  0.034918       1        1  58.572902
8     -0.199849 -0.125110       2        1  23.666310
0     -0.141409  0.098980       3        1  10.383597
14     0.002974 -0.005858       4        1   1.512415
6      0.000438  0.025006       5        1   1.118556
13     0.050892 -0.016945       6        1   0.834121
5      0.040760  0.007667       7        1   0.684973
12     0.069626 -0.010073       8        1   0.597095
3      0.057155  0.001486       9        1   0.572990
9      0.064220  0.002871      10        1   0.428719
4      0.064905 -0.003057      11        1   0.398169
2      0.064397 -0.002999      12        1   0.330472
10     0.064506 -0.001004      13        1   0.323522
11     0.062847 -0.004914      14        1   0.299008
1      0.061815 -0.000967      15        1   0.277151, topic_info=          Term        Freq       Total Category  logprob  loglift
52        wing  589.000000  589.000000  Default  30.0000  30.0000
55        bodi  538.000000  538.000000  Default  29.0000  29.0000
93       shock  475.000000  475.000000  Default  28.0000  28.0000
335   transfer  361.000000  361.000000  Default  27.0000  27.0000
70    hyperson  307.000000  307.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
1463   preston    0.544964   16.884500  Topic15  -6.3889   2.4549
2245   entranc    0.409394    6.658301  Topic15  -6.6749   3.0994
498      circl    0.489370   20.486858  Topic15  -6.4965   2.1539
2256      eros    0.448487   14.102088  Topic15  -6.5837   2.4402
174       inch    0.448106   28.419677  Topic15  -6.5846   1.7386

[752 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1744      1  0.187102  accommod
1744      2  0.187102  accommod
1744      3  0.187102  accommod
1744     12  0.187102  accommod
1804      1  0.169358   accumul
...     ...       ...       ...
2387     10  0.193797     young
2351      1  0.215433      zoom
2351      2  0.215433      zoom
2351      3  0.215433      zoom
2351     10  0.215433      zoom

[2839 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 9, 1, 15, 7, 14, 6, 13, 4, 10, 5, 3, 11, 12, 2])

**Performance Check of the model with n=12 and n=15 on the unseen data**

n=12

In [ ]:
unseen_data1 = "Aerodynamic interaction between propellers of a distributed-propulsion system in forward flight"
corpus1 = dictionary.doc2bow(preprocess(unseen_data1))
for index, score in sorted(lda2[corpus1], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda2.print_topic(index, 2)))

Score: 0.7750461101531982	 Topic: 0.007*"layer" + 0.007*"boundari"
Score: 0.12078623473644257	 Topic: 0.009*"flap" + 0.008*"tail"
Score: 0.010417289100587368	 Topic: 0.018*"wing" + 0.009*"flutter"
Score: 0.010416824370622635	 Topic: 0.004*"charg" + 0.004*"particl"
Score: 0.010416782461106777	 Topic: 0.004*"pohlhausen" + 0.004*"fluctuat"
Score: 0.010416774079203606	 Topic: 0.022*"buckl" + 0.018*"shell"
Score: 0.010416670702397823	 Topic: 0.011*"fatigu" + 0.006*"nois"
Score: 0.0104166679084301	 Topic: 0.003*"eddi" + 0.003*"orbit"
Score: 0.0104166679084301	 Topic: 0.004*"entropi" + 0.004*"creep"
Score: 0.0104166679084301	 Topic: 0.003*"water" + 0.003*"dash"
Score: 0.0104166679084301	 Topic: 0.012*"compressor" + 0.008*"orbit"
Score: 0.0104166679084301	 Topic: 0.003*"frustum" + 0.002*"sound"


n=15

In [ ]:
unseen_data2 = "Aerodynamic interaction between propellers of a distributed-propulsion system in forward flight"
corpus2 = dictionary.doc2bow(preprocess(unseen_data2))
for index, score in sorted(lda6[corpus2], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda6.print_topic(index, 2)))

Score: 0.6169230341911316	 Topic: 0.010*"buckl" + 0.009*"shell"
Score: 0.2747434973716736	 Topic: 0.008*"flow" + 0.008*"wing"


n=12

In [ ]:
unseen_data3 = "Turbulence Model Could Help Design Aircraft Capable of Handling Extreme Scenarios"
corpus3 = dictionary.doc2bow(preprocess(unseen_data3))
for index, score in sorted(lda2[corpus3], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda2.print_topic(index, 2)))

Score: 0.7126365303993225	 Topic: 0.007*"layer" + 0.007*"boundari"
Score: 0.19476987421512604	 Topic: 0.018*"wing" + 0.009*"flutter"


n=15

In [ ]:
unseen_data4 = "Turbulence Model Could Help Design Aircraft Capable of Handling Extreme Scenarios"
corpus4 = dictionary.doc2bow(preprocess(unseen_data4))
for index, score in sorted(lda6[corpus4], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda6.print_topic(index, 2)))

Score: 0.6537162065505981	 Topic: 0.008*"flow" + 0.008*"wing"
Score: 0.24998739361763	 Topic: 0.010*"buckl" + 0.009*"shell"


n=12

In [ ]:
unseen_data5 = "Zero emissions hydrogen plane test was part powered by fossil fuels"
corpus5 = dictionary.doc2bow(preprocess(unseen_data5))
for index, score in sorted(lda2[corpus5], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda2.print_topic(index, 2)))

Score: 0.7604056000709534	 Topic: 0.007*"layer" + 0.007*"boundari"
Score: 0.13542647659778595	 Topic: 0.012*"compressor" + 0.008*"orbit"
Score: 0.010417105630040169	 Topic: 0.018*"wing" + 0.009*"flutter"
Score: 0.010416975244879723	 Topic: 0.022*"buckl" + 0.018*"shell"
Score: 0.010416939854621887	 Topic: 0.004*"charg" + 0.004*"particl"
Score: 0.010416872799396515	 Topic: 0.003*"water" + 0.003*"dash"
Score: 0.010416730307042599	 Topic: 0.003*"frustum" + 0.002*"sound"
Score: 0.010416669771075249	 Topic: 0.004*"pohlhausen" + 0.004*"fluctuat"
Score: 0.0104166679084301	 Topic: 0.011*"fatigu" + 0.006*"nois"
Score: 0.0104166679084301	 Topic: 0.009*"flap" + 0.008*"tail"
Score: 0.010416666977107525	 Topic: 0.003*"eddi" + 0.003*"orbit"
Score: 0.010416666977107525	 Topic: 0.004*"entropi" + 0.004*"creep"


n=16

In [ ]:
unseen_data6 = "Zero emissions hydrogen plane test was part powered by fossil fuels"
corpus6 = dictionary.doc2bow(preprocess(unseen_data6))
for index, score in sorted(lda6[corpus6], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda6.print_topic(index, 2)))

Score: 0.41834452748298645	 Topic: 0.008*"flow" + 0.008*"wing"
Score: 0.34831538796424866	 Topic: 0.011*"compressor" + 0.007*"orbit"
Score: 0.13333997130393982	 Topic: 0.006*"prescrib" + 0.004*"vehicl"


n=12

In [ ]:
unseen_data7 = "how can one detect transition phenomena in boundary layers"
corpus7 = dictionary.doc2bow(preprocess(unseen_data7))
for index, score in sorted(lda2[corpus7], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda2.print_topic(index, 2)))

Score: 0.6726142764091492	 Topic: 0.007*"layer" + 0.007*"boundari"
Score: 0.1884964555501938	 Topic: 0.004*"pohlhausen" + 0.004*"fluctuat"
Score: 0.013889055699110031	 Topic: 0.018*"wing" + 0.009*"flutter"
Score: 0.013889019377529621	 Topic: 0.003*"water" + 0.003*"dash"
Score: 0.013888935558497906	 Topic: 0.022*"buckl" + 0.018*"shell"
Score: 0.013888920657336712	 Topic: 0.004*"charg" + 0.004*"particl"
Score: 0.013888892717659473	 Topic: 0.011*"fatigu" + 0.006*"nois"
Score: 0.013888888992369175	 Topic: 0.003*"eddi" + 0.003*"orbit"
Score: 0.013888888992369175	 Topic: 0.004*"entropi" + 0.004*"creep"
Score: 0.013888888992369175	 Topic: 0.012*"compressor" + 0.008*"orbit"
Score: 0.013888888992369175	 Topic: 0.003*"frustum" + 0.002*"sound"
Score: 0.013888888992369175	 Topic: 0.009*"flap" + 0.008*"tail"


In [ ]:
unseen_data8 = "No One Can Explain Why Planes Stay in the Air"
corpus8 = dictionary.doc2bow(preprocess(unseen_data8))
for index, score in sorted(lda2[corpus8], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda2.print_topic(index, 2)))

Score: 0.6944437623023987	 Topic: 0.018*"wing" + 0.009*"flutter"
Score: 0.02777812071144581	 Topic: 0.003*"eddi" + 0.003*"orbit"
Score: 0.027777912095189095	 Topic: 0.004*"pohlhausen" + 0.004*"fluctuat"
Score: 0.027777863666415215	 Topic: 0.022*"buckl" + 0.018*"shell"
Score: 0.027777837589383125	 Topic: 0.007*"layer" + 0.007*"boundari"
Score: 0.027777792885899544	 Topic: 0.009*"flap" + 0.008*"tail"
Score: 0.02777778171002865	 Topic: 0.012*"compressor" + 0.008*"orbit"
Score: 0.02777778171002865	 Topic: 0.011*"fatigu" + 0.006*"nois"
Score: 0.02777777798473835	 Topic: 0.004*"charg" + 0.004*"particl"
Score: 0.02777777798473835	 Topic: 0.004*"entropi" + 0.004*"creep"
Score: 0.02777777798473835	 Topic: 0.003*"water" + 0.003*"dash"
Score: 0.02777777798473835	 Topic: 0.003*"frustum" + 0.002*"sound"


n=15

In [ ]:
unseen_data9 = "No One Can Explain Why Planes Stay in the Air"
corpus9 = dictionary.doc2bow(preprocess(unseen_data9))
for index, score in sorted(lda6[corpus9], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda6.print_topic(index, 2)))

Score: 0.35555583238601685	 Topic: 0.011*"inject" + 0.011*"transfer"
Score: 0.35555514693260193	 Topic: 0.008*"flow" + 0.008*"wing"
Score: 0.022222332656383514	 Topic: 0.010*"buckl" + 0.009*"shell"
Score: 0.02222222462296486	 Topic: 0.003*"corridor" + 0.002*"raleigh"
Score: 0.02222222089767456	 Topic: 0.003*"heater" + 0.003*"liapunov"
Score: 0.02222222089767456	 Topic: 0.005*"freon" + 0.003*"comment"
Score: 0.02222222089767456	 Topic: 0.006*"prescrib" + 0.004*"vehicl"
Score: 0.02222222089767456	 Topic: 0.003*"multi" + 0.003*"theodorsen"
Score: 0.02222222089767456	 Topic: 0.005*"exit" + 0.005*"converg"
Score: 0.02222222089767456	 Topic: 0.005*"blast" + 0.005*"hyperson"
Score: 0.02222222089767456	 Topic: 0.005*"freestream" + 0.003*"con"
Score: 0.02222222089767456	 Topic: 0.003*"hodograph" + 0.003*"vane"
Score: 0.02222222089767456	 Topic: 0.007*"blade" + 0.006*"impel"
Score: 0.02222222089767456	 Topic: 0.009*"slipstream" + 0.008*"tilt"
Score: 0.02222222089767456	 Topic: 0.011*"compressor"